In [4]:
from pyproj import Transformer
import pandas as pd

In [26]:
# 排除公園 'POI_LON', 'POI_LAT' 重複值
df = pd.read_csv("公園.csv")

# 將POI_YM列轉換為整數型別
df['POI_YM'] = df['POI_YM'].astype(int)

# 根據POI_YM列的值進行分組，保留POI_YM最大的數值資料，並刪除其他重複的數據
df = df.sort_values(by='POI_YM', ascending=False).drop_duplicates(subset=['POI_LON', 'POI_LAT'])

# 將處理後的DataFrame保存回CSV文件
df.to_csv("公園_new.csv", index=False)

print("公園_new.csv文件")


已保存處理後的公園_data.csv文件


In [27]:
# 排除垃圾場 'POI_LON', 'POI_LAT' 重複值
df_2 = pd.read_csv("垃圾場.csv")

# 將POI_YM列轉換為整數型別
df_2['POI_YM'] = df_2['POI_YM'].astype(int)

# 根據POI_YM列的值進行分組，保留POI_YM最大的數值資料，並刪除其他重複的數據
df_2 = df_2.sort_values(by='POI_YM', ascending=False).drop_duplicates(subset=['POI_LON', 'POI_LAT'])

# 將處理後的DataFrame保存回CSV文件
df_2.to_csv("垃圾場_new.csv", index=False)

print("垃圾場_new.csv文件")


垃圾場_new.csv文件


In [32]:
# 排除產業工業園區 'POI_LON', 'POI_LAT' 重複值
df_3 = pd.read_csv("產業工業園區.csv")

# 將POI_YM列轉換為整數型別
df_3['POI_YM'] = df_3['POI_YM'].astype(int)

# 根據POI_YM列的值進行分組，保留POI_YM最大的數值資料，並刪除其他重複的數據
df_3 = df_3.sort_values(by='POI_YM', ascending=False).drop_duplicates(subset=['POI_LON', 'POI_LAT'])

# 將處理後的DataFrame保存回CSV文件
df_3.to_csv("產業工業園區_new.csv", index=False)

print("產業工業園區_new.csv文件")


產業工業園區_new.csv文件


In [33]:
# 排除焚化廠 'POI_LON', 'POI_LAT' 重複值
df_4 = pd.read_csv("焚化廠.csv")

# 將POI_YM列轉換為整數型別
df_4['POI_YM'] = df_4['POI_YM'].astype(int)

# 根據POI_YM列的值進行分組，保留POI_YM最大的數值資料，並刪除其他重複的數據
df_4 = df_4.sort_values(by='POI_YM', ascending=False).drop_duplicates(subset=['POI_LON', 'POI_LAT'])

# 將處理後的DataFrame保存回CSV文件
df_4.to_csv("焚化廠_new.csv", index=False)

print("焚化廠_new.csv文件")


焚化廠_new.csv文件


In [35]:
# 排除殯葬設施 'POI_LON', 'POI_LAT' 重複值
df_5 = pd.read_csv("殯葬設施.csv")

# 將POI_YM列轉換為整數型別
df_5['POI_YM'] = df_5['POI_YM'].astype(int)

# 根據POI_YM列的值進行分組，保留POI_YM最大的數值資料，並刪除其他重複的數據
df_5 = df_5.sort_values(by='POI_YM', ascending=False).drop_duplicates(subset=['POI_LON', 'POI_LAT'])

# 將處理後的DataFrame保存回CSV文件
df_5.to_csv("殯葬設施_new.csv", index=False)

print("殯葬設施_new.csv文件")


殯葬設施_new.csv文件


In [43]:
converted_data = pd.read_csv("converted_data_new_taipei_city.csv",encoding='utf-8-sig',dtype={"公園": str})
print(converted_data)

             Lon1       Lat1        Lon2       Lat2   X   Y   公園  產業工業園區  垃圾場  \
0      121.457125  25.105445  121.458119  25.106345 NaN NaN  NaN     NaN  NaN   
1      121.457128  25.106348  121.458123  25.107248 NaN NaN  NaN     NaN  NaN   
2      121.457131  25.107251  121.458126  25.108151 NaN NaN  NaN     NaN  NaN   
3      121.457135  25.108154  121.458129  25.109054 NaN NaN  NaN     NaN  NaN   
4      121.457138  25.109057  121.458133  25.109956 NaN NaN  NaN     NaN  NaN   
...           ...        ...         ...        ...  ..  ..  ...     ...  ...   
27656  121.664917  25.027019  121.665913  25.027918 NaN NaN  NaN     NaN  NaN   
27657  121.664922  25.027922  121.665918  25.028820 NaN NaN  NaN     NaN  NaN   
27658  121.664927  25.028825  121.665922  25.029723 NaN NaN  NaN     NaN  NaN   
27659  121.664931  25.029728  121.665927  25.030626 NaN NaN  NaN     NaN  NaN   
27660  121.665922  25.029723  121.666918  25.030622 NaN NaN  NaN     NaN  NaN   

       焚化廠  殯葬設施  
0      N

In [ ]:
#區域發展指標統計_版本1
import pandas as pd
# 讀取 converted_data_taipei.csv 檔案並指定 'Store_Name' 欄位的資料類型為字串
converted_data_df = pd.read_csv("converted_data_new_taipei_city.csv", dtype={"E": str})

# 讀取 公園_data.xlsx 檔案
geocode_df = pd.read_csv("殯葬設施_new.csv", encoding='utf-8-sig')

# 定義尋找對應方格索引的函數
def find_point(row):
    x = row['POI_LON']
    y = row['POI_LAT']
    for index, point_row in converted_data_df.iterrows():
        if (point_row['Lon1'] <= x <= point_row['Lon2'] or
            point_row['Lon2'] <= x <= point_row['Lon1']) and \
           (point_row['Lat1'] <= y <= point_row['Lat2'] or
            point_row['Lat2'] <= y <= point_row['Lat1']):
            return index
    return None

# 將 geocode_df 中的 X、Y 新增到 converted_data_df 對應方格的 X、Y 欄位中
for index, row in geocode_df.iterrows():
    point_index = find_point(row)
    if point_index is not None:
        converted_data_df.at[point_index, 'X'] = row['POI_LON']
        converted_data_df.at[point_index, 'Y'] = row['POI_LAT']
         # 獲取已存在的 POI_ID
        existing_poi_id = converted_data_df.at[point_index, 'E']
        # 如果已存在的 POI_ID 不為空，則將新的 POI_ID 附加到已存在的 POI_ID 後面，並使用分號分隔
        if pd.notnull(existing_poi_id):
            new_poi_id = f"{existing_poi_id};{row['POI_ID']}"
        else:
            new_poi_id = row['POI_ID']
        converted_data_df.at[point_index, 'E'] = new_poi_id
        
# 將結果寫入 converted_data_taipei.csv 檔案中
converted_data_df.to_csv("converted_data_new_taipei_city.csv", index=False,encoding='utf-8-sig')

print("已寫入 converted_data_new_taipei_city.csv 殯葬設施 檔案中")

In [ ]:
#區域發展指標統計_版本2

import pandas as pd

# 讀取 converted_data_taipei.csv 檔案並指定 'Store_Name' 欄位的資料類型為字串
converted_data_df = pd.read_csv("converted_data_new_taipei_city.csv", dtype={"A": str})

# 讀取 公園_data.xlsx 檔案
geocode_df = pd.read_csv("公園_new.csv", encoding='utf-8-sig')

# 定義尋找對應方格索引的函數
def find_point(row, converted_data_df):
    x = row['POI_LON']
    y = row['POI_LAT']
    
    # 使用向量化操作來找到對應方格的索引
    index = ((converted_data_df['Lon1'] <= x) & (x <= converted_data_df['Lon2']) | 
             (converted_data_df['Lon2'] <= x) & (x <= converted_data_df['Lon1'])) & \
            ((converted_data_df['Lat1'] <= y) & (y <= converted_data_df['Lat2']) | 
             (converted_data_df['Lat2'] <= y) & (y <= converted_data_df['Lat1']))
    
    if index.any():
        return index.idxmax()
    else:
        return None

# 將 geocode_df 中的 X、Y 新增到 converted_data_df 對應方格的 X、Y 欄位中
geocode_df['point_index'] = geocode_df.apply(lambda row: find_point(row, converted_data_df), axis=1)

# 將找到的 POI_ID 添加到 converted_data_df 中
for _, row in geocode_df.iterrows():
    if not pd.isnull(row['point_index']):  # 檢查 point_index 是否為 NaN
        # 獲取已存在的 POI_ID
        existing_poi_id = converted_data_df.at[row['point_index'], 'A']
        # 如果已存在的 POI_ID 不為空，則將新的 POI_ID 附加到已存在的 POI_ID 後面，並使用分號分隔
        if pd.notnull(existing_poi_id):
            new_poi_id = f"{existing_poi_id};{row['POI_ID']}"
        else:
            new_poi_id = row['POI_ID']
        converted_data_df.at[row['point_index'], 'A'] = new_poi_id
        # 更新 X、Y 欄位
        converted_data_df.at[row['point_index'], 'X'] = row['POI_LON']
        converted_data_df.at[row['point_index'], 'Y'] = row['POI_LAT']

# 將結果寫入 converted_data_taipei.csv 檔案中
converted_data_df.to_csv("converted_data_new_taipei_city.csv", index=False, encoding='utf-8-sig')

print("已寫入 converted_data_new_taipei_city.csv 公園 檔案中")